##**installing pytorch and detectron2** 

In [ ]:
!pip install torch==1.10.1+cu111 torchvision==0.11.2+cu111 torchaudio==0.10.1 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
!pip install detectron2==0.6 -f \
https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.10/index.html

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torchvision
import torch
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)
print(torch.__version__, torch.cuda.is_available())

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer , ColorMode, _create_text_labels
from detectron2.utils.colormap import random_color

from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0
torch:  1.10 ; cuda:  cu111
detectron2: 0.6
1.10.1+cu111 True


In [3]:
import cv2 as cv2
import os , random
import matplotlib.pyplot as plt
from PIL import Image
import PIL 
import numpy as np
import json
import glob
import pandas as pd
from google.colab.patches import cv2_imshow

In [4]:
os.chdir("/content/drive/MyDrive/pharmacy/")

In [5]:
os.getcwd()

'/content/drive/.shortcut-targets-by-id/1of130zC4WOxL1MV2r7snVPEROqNr9fgT/pharmacy'

In [ ]:
DatasetCatalog.clear()

## **Register dataset**

In [6]:
def get_dataset_dicts(p):
  with open(p+".json","r") as f:
    data_dict = json.load(f)
  return data_dict

In [ ]:
d="syn_testset"
DatasetCatalog.register( d, lambda d=d: get_dataset_dicts("./data/" + d))
MetadataCatalog.get( d).set(thing_classes=['cycloff', 'bibron', 'cortéma', 'laroche_posay','soulagel', 'sucette'])
test_metadata = MetadataCatalog.get("syn_testset")

In [ ]:
for d in ["synthetic_train", "synthetic_test"]:
    DatasetCatalog.register("" + d, lambda d=d: get_dataset_dicts("./data/" + d))
    MetadataCatalog.get("" + d).set(thing_classes=['cycloff', 'bibron', 'cortéma', 'laroche_posay','soulagel', 'sucette'])
train_metadata = MetadataCatalog.get("synthetic_train")

In [ ]:
#data_train_name = "synthetic_train"
#data_test_name ="synthetic_test"
#data_test_name ="test"
data_test_name ="syn_testset"

In [ ]:
cfg = get_cfg()
cfg.OUTPUT_DIR = "./logs"

cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
#cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
#cfg.MODEL.WEIGHTS = "./model_0005247.pth"
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster (default: 512)
#cfg.DATASETS.TRAIN = (data_train_name,)
cfg.DATASETS.TEST = (data_test_name,)
# ("fashion_train",)
cfg.INPUT.MAX_SIZE_TRAIN = 800
cfg.INPUT.MIN_SIZE_TRAIN = 800

cfg.INPUT.MIN_SIZE_TEST = 800
# Maximum size of the side of the image during testing
cfg.INPUT.MAX_SIZE_TEST = 800

cfg.DATALOADER.NUM_WORKERS = 4

cfg.SOLVER.BASE_LR = 0.001  
# pick a good LR

cfg.SOLVER.MAX_ITER = 63012 #21471 #35693 #142075  #2160
# The iteration number to decrease learning rate by GAMMA. to decay lr (12,17)
cfg.SOLVER.STEPS = (58000,)
cfg.SOLVER.gamma = 0.5
cfg.SOLVER.IMS_PER_BATCH = 8
# after each epc        # do not decay learning rate
cfg.SOLVER.CHECKPOINT_PERIOD =  630 #356 #1420 #215 #  36*2

#cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 6
#cfg.TEST.EVAL_PERIOD = 2840
#cfg.TEST.DETECTIONS_PER_IMAGE = 50

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
cfg.MODEL.DEVICE = "cuda"

In [ ]:
train_metadata = MetadataCatalog.get(data_train_name)

In [ ]:
test_metadata = MetadataCatalog.get(data_test_name)

In [ ]:
dict_test[0]

{'file_name': './data/test_synthetic/1000.jpg',
 'height': 2100,
 'width': 2400,
 'image_id': 1000,
 'annotations': [{'iscrowd': 0,
   'bbox': [332, 319, 1869, 1770],
   'category_id': 1,
   'bbox_mode': 0}]}

In [ ]:
with open('/content/drive/MyDrive/pharmacy/data/synthetic_test.json','w') as ap:
  json.dump(dict_test,ap)

In [ ]:
len(dict_test)

26

In [ ]:
#train_metadata.thing_classes

['cycloff', 'bibron', 'cortéma', 'laroche_posay', 'soulagel', 'sucette']

In [ ]:
len(DatasetCatalog.get(data_test_name))

464

In [ ]:
detectron2.data.print_instances_class_histogram(dict_test, test_metadata.thing_classes)

[10/21 07:32:29 d2.data.build]: Distribution of instances among all 6 categories:
|   category    | #instances   |  category  | #instances   |  category  | #instances   |
|:-------------:|:-------------|:----------:|:-------------|:----------:|:-------------|
|    cycloff    | 10           |   bibron   | 28           |  cortéma   | 25           |
| laroche_posay | 45           |  soulagel  | 35           |  sucette   | 10           |
|               |              |            |              |            |              |
|     total     | 153          |            |              |            |              |


In [ ]:
detectron2.data.print_instances_class_histogram(dict_train, train_metadata.thing_classes)

[10/20 00:10:12 d2.data.build]: Distribution of instances among all 6 categories:
|   category    | #instances   |  category  | #instances   |  category  | #instances   |
|:-------------:|:-------------|:----------:|:-------------|:----------:|:-------------|
|    cycloff    | 334          |   bibron   | 802          |  cortéma   | 838          |
| laroche_posay | 1463         |  soulagel  | 1194         |  sucette   | 394          |
|               |              |            |              |            |              |
|     total     | 5025         |            |              |            |              |


In [ ]:
it_epoch = 5025/8
print(it_epoch+2)
max_it = (it_epoch+2) * 100
print(max_it)
print(max_it/2)

630.125
63012.5
31506.25


In [ ]:
#for i in range(len(dict_train)):
#  im = Image.open(dict_train[i]['file_name'])
#  width, height = im.size
#  dict_train[i]['height'] = height
#  dict_train[i]['width'] = width

In [ ]:
evaluator = COCOEvaluator(data_test_name,tasks=("bbox",),output_dir="./")

[10/12 07:43:08 d2.evaluation.coco_evaluation]: Trying to convert 'data_test' to COCO format ...
[10/12 07:43:08 d2.data.datasets.coco]: Converting annotations of dataset 'data_test' to COCO format ...)
[10/12 07:43:08 d2.data.datasets.coco]: Converting dataset dicts into COCO format
[10/12 07:43:08 d2.data.datasets.coco]: Conversion finished, #images: 54, #annotations: 54
[10/12 07:43:08 d2.data.datasets.coco]: Caching COCO format annotations at './data_test_coco_format.json' ...


In [ ]:
!nvidia-smi

Thu Oct 20 05:50:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

 ## **simple training**

In [ ]:
trainer = DefaultTrainer(cfg)
#model = trainer.build_model(cfg)
#trainer.test(cfg, model,evaluators=evaluator)
trainer.resume_or_load(resume = False) # load last checkpoint or MODEL.WEIGHTS
#trainer.build_evaluator 
trainer.train()

##**Data augmenattion**


In [ ]:
from detectron2.data import detection_utils as utils
import detectron2.data.transforms as T
import copy

def custom_mapper(dataset_dict):
    dataset_dict = copy.deepcopy(dataset_dict)  # it will be modified by code below
    image = utils.read_image(dataset_dict["file_name"], format="BGR")
    transform_list = [
        T.Resize((800,800)),
       #T.RandomRotation(angle=[20,60],expand=True, center=[[300,300],[465,1600]],sample_style='range'),
      #  T.RandomFlip(prob=0.2, horizontal=False, vertical=True),
    ]
    image, transforms = T.apply_transform_gens(transform_list, image)
    dataset_dict["image"] = torch.as_tensor(image.transpose(2, 0, 1).astype("float32"))

    annos = [
        utils.transform_instance_annotations(obj, transforms, image.shape[:2])
        for obj in dataset_dict.pop("annotations")
        if obj.get("iscrowd", 0) == 0
    ]
    instances = utils.annotations_to_instances(annos, image.shape[:2])
    dataset_dict["instances"] = utils.filter_empty_instances(instances)
    return dataset_dict

**custom training**

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.data import build_detection_test_loader, build_detection_train_loader

class CustomTrainer(DefaultTrainer):
    @classmethod
    def build_train_loader(cls, cfg):
        return build_detection_train_loader(cfg, mapper=custom_mapper)
#%%
trainer = CustomTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

## Test

#### custom mapper

In [ ]:
####### Sauter #######
####### Data augmentation ##############
from detectron2.data import detection_utils as utils
import detectron2.data.transforms as T
#import detectron2.data.transforms.Augmentation as A
import copy
def custom_mapper(dataset_dict, pad, angle):
    dataset_dict = copy.deepcopy(dataset_dict)  # it will be modified by code below
    image = utils.read_image(dataset_dict["file_name"], format="BGR")
    w,h = image.shape[:2]
    transform_list = [
        T.NoOpTransform(),
        #T.RandomFlip(prob=1, horizontal=False, vertical=True),
        #T.RandomRotation(angle=[angle],expand=True,sample_style='choice', center=None),
        #T.PadTransform(0,0,pad,pad,0),
        #T.Resize((h,w)),
        #T.RandomLighting(0.5),
        ]
        #T.RandomFlip(prob=0.7, horizontal=False, vertical=True),
       # T.RandomFlip(prob=0.7, horizontal=True, vertical=False),]
   
    image, transforms = T.apply_transform_gens(transform_list, image)
    dataset_dict["image"] = torch.as_tensor(image.transpose(2, 0, 1).astype("float32"))

    annos = [
        utils.transform_instance_annotations(obj, transforms, image.shape[:2])
        for obj in dataset_dict.pop("annotations")
        if obj.get("iscrowd", 0) == 0
    ]
    instances = utils.annotations_to_instances(annos, image.shape[:2])
    dataset_dict["instances"] = utils.filter_empty_instances(instances)
    return dataset_dict , image